# Bipartite Link Prediction

## Introduction

Having previously explored link prediction in monopartite networks, now we move on to the more complex scenario where we have a bipartite network. The goal will be to generalize the monopartite link prediction methods to the bipartite case, which will require us to redefine some key quantities. For instance, there can be no such thing as a common neighbor in a bipartite network, since nodes of the same class cannot connect to each other directly. The bipartite analog of common neighbors developed by Cannistraci is the concept of local community links (LCLs), which are the links between the first degree neighbors of two nodes. Similarly, the clustering coefficient had to be redefined in terms of the number of quadrangles formed between two nodes instead of the number of triangles.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import b_datasets as ds
import b_val as val
import b_lcp
import b_mi
import b_si
import b_probas as probas

In [3]:
fraction = 0.1
loops = 100
verbose=True
plot=False

## Load datasets

The validation process is the same as for monopartite networks, where we randomly delete a given fraction of the links and then attempt to recover them, averaging the results over several trials. Again we create a list of links to delete beforehand to ensure reproducibility and consistency between different methods in this notebook. 

In [4]:
# Elegans data (multipartite)
s_nt, nt_nr, nr_t = ds.load_elegans_data()

In [5]:
# Giant component of s_nt
s_nt_giant = ds.giant_component_bp(s_nt)

In [6]:
# Other datasets (bipartite only)
x_e = ds.load_enzyme_data()
x_g = ds.load_gpcr_data()
x_ic = ds.load_ion_channel_data()

In [7]:
# Precompute links to delete for all datasets
links_to_del_s_nt = val.get_links_to_del(s_nt, fraction=fraction, loops=loops)
links_to_del_nt_nr = val.get_links_to_del(nt_nr, fraction=fraction, loops=loops)
links_to_del_nr_t = val.get_links_to_del(nr_t, fraction=fraction, loops=loops)

In [8]:
links_to_del_s_nt_giant = val.get_links_to_del(s_nt_giant, fraction=fraction, loops=loops)

In [9]:
links_to_del_e = val.get_links_to_del(x_e, fraction=fraction, loops=loops)
links_to_del_g = val.get_links_to_del(x_g, fraction=fraction, loops=loops)
links_to_del_ic = val.get_links_to_del(x_ic, fraction=fraction, loops=loops)

## Reproduce top Cannistraci results

As a check, we can first reproduce some results from the Cannistraci paper (Daminelli et al, New Journal of Phys 2015) where they introduce the concept of the bipartite local community paradigm. 

In [11]:
# Enzymes - looking for about 0.6 AUPR
r = val.cross_val(b_lcp.bipartite_lcp_single, x_e, links_to_del_e, *('cra',), 
                  mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r, axis=0))
print(np.std(r, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.98 %, 0.43 minutes
Trial 2 of 100
Computing bipartite LCP
99.98 %, 0.44 minutes
Trial 3 of 100
Computing bipartite LCP
99.98 %, 0.43 minutes
Trial 4 of 100
Computing bipartite LCP
99.98 %, 0.43 minutes
Trial 5 of 100
Computing bipartite LCP
99.98 %, 0.39 minutes
Trial 6 of 100
Computing bipartite LCP
99.98 %, 0.37 minutes
Trial 7 of 100
Computing bipartite LCP
99.98 %, 0.38 minutes
Trial 8 of 100
Computing bipartite LCP
99.98 %, 0.37 minutes
Trial 9 of 100
Computing bipartite LCP
99.98 %, 0.37 minutes
Trial 10 of 100
Computing bipartite LCP
99.98 %, 0.38 minutes
Trial 11 of 100
Computing bipartite LCP
99.98 %, 0.38 minutes
Trial 12 of 100
Computing bipartite LCP
99.98 %, 0.38 minutes
Trial 13 of 100
Computing bipartite LCP
99.98 %, 0.37 minutes
Trial 14 of 100
Computing bipartite LCP
99.98 %, 0.38 minutes
Trial 15 of 100
Computing bipartite LCP
99.98 %, 0.37 minutes
Trial 16 of 100
Computing bipartite LCP
99.98 %, 0.38 minutes
Trial 17 of 100
C

In [12]:
# GPCR - looking for about 0.3 AUPR
r1 = val.cross_val(b_lcp.bipartite_lcp_single, x_g, links_to_del_g, *('cra',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r1, axis=0))
print(np.std(r1, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.94 %, 0.07 minutes
Trial 2 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 3 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 4 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 5 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 6 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 7 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 8 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 9 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 10 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 11 of 100
Computing bipartite LCP
99.94 %, 0.07 minutes
Trial 12 of 100
Computing bipartite LCP
99.94 %, 0.11 minutes
Trial 13 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 14 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 15 of 100
Computing bipartite LCP
99.94 %, 0.06 minutes
Trial 16 of 100
Computing bipartite LCP
99.94 %, 0.05 minutes
Trial 17 of 100
C

In [13]:
# Ion Channels - looking for about 0.5 AUPR
r2 = val.cross_val(b_lcp.bipartite_lcp_single, x_ic, links_to_del_ic, *('cjc',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r2, axis=0))
print(np.std(r2, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.97 %, 0.17 minutes
Trial 2 of 100
Computing bipartite LCP
99.97 %, 0.16 minutes
Trial 3 of 100
Computing bipartite LCP
99.97 %, 0.17 minutes
Trial 4 of 100
Computing bipartite LCP
99.97 %, 0.16 minutes
Trial 5 of 100
Computing bipartite LCP
99.97 %, 0.17 minutes
Trial 6 of 100
Computing bipartite LCP
99.97 %, 0.27 minutes
Trial 7 of 100
Computing bipartite LCP
99.97 %, 0.16 minutes
Trial 8 of 100
Computing bipartite LCP
99.97 %, 0.16 minutes
Trial 9 of 100
Computing bipartite LCP
99.97 %, 0.16 minutes
Trial 10 of 100
Computing bipartite LCP
99.97 %, 0.16 minutes
Trial 11 of 100
Computing bipartite LCP
99.97 %, 0.16 minutes
Trial 12 of 100
Computing bipartite LCP
99.97 %, 0.17 minutes
Trial 13 of 100
Computing bipartite LCP
99.97 %, 0.16 minutes
Trial 14 of 100
Computing bipartite LCP
99.97 %, 0.16 minutes
Trial 15 of 100
Computing bipartite LCP
99.97 %, 0.15 minutes
Trial 16 of 100
Computing bipartite LCP
99.97 %, 0.15 minutes
Trial 17 of 100
C

## Now try LCP on the Elegans data

The bipartite LCP methods appear to work reasonably well on the above datasets, but unfortunately the Elegans dataset has proved to be much more difficult to work with.

In [14]:
# CRA does very well on two of the other datasets
r3 = val.cross_val(b_lcp.bipartite_lcp_single, s_nt, links_to_del_s_nt, *('cra',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r3, axis=0))
print(np.std(r3, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 2 of 100
Computing bipartite LCP
99.54 %, 0.14 minutes
Trial 3 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 4 of 100
Computing bipartite LCP
99.54 %, 0.14 minutes
Trial 5 of 100
Computing bipartite LCP
99.54 %, 0.08 minutes
Trial 6 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 7 of 100
Computing bipartite LCP
99.54 %, 0.07 minutes
Trial 8 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 9 of 100
Computing bipartite LCP
99.54 %, 0.09 minutes
Trial 10 of 100
Computing bipartite LCP
99.54 %, 0.10 minutes
Trial 11 of 100
Computing bipartite LCP
99.54 %, 0.08 minutes
Trial 12 of 100
Computing bipartite LCP
99.54 %, 0.07 minutes
Trial 13 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 14 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 15 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 16 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 17 of 100
C

In [15]:
# So does CJC
r4 = val.cross_val(b_lcp.bipartite_lcp_single, s_nt, links_to_del_s_nt, *('cjc',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r4, axis=0))
print(np.std(r4, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.54 %, 0.08 minutes
Trial 2 of 100
Computing bipartite LCP
99.54 %, 0.08 minutes
Trial 3 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 4 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 5 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 6 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 7 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 8 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 9 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 10 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 11 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 12 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 13 of 100
Computing bipartite LCP
99.54 %, 0.08 minutes
Trial 14 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 15 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 16 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 17 of 100
C

In [16]:
# And let's try LCL since its one of the simplest
r5 = val.cross_val(b_lcp.bipartite_lcp_single, s_nt, links_to_del_s_nt, *('lcl',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r5, axis=0))
print(np.std(r5, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.54 %, 0.07 minutes
Trial 2 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 3 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 4 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 5 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 6 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 7 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 8 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 9 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 10 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 11 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 12 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 13 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 14 of 100
Computing bipartite LCP
99.54 %, 0.08 minutes
Trial 15 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 16 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 17 of 100
C

In [17]:
# Now try with the giant component to see if that makes a difference
r6 = val.cross_val(b_lcp.bipartite_lcp_single, s_nt_giant, links_to_del_s_nt_giant, *('lcl',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r6, axis=0))
print(np.std(r6, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.52 %, 0.06 minutes
Trial 2 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 3 of 100
Computing bipartite LCP
99.52 %, 0.06 minutes
Trial 4 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 5 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 6 of 100
Computing bipartite LCP
99.52 %, 0.06 minutes
Trial 7 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 8 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 9 of 100
Computing bipartite LCP
99.52 %, 0.06 minutes
Trial 10 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 11 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 12 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 13 of 100
Computing bipartite LCP
99.52 %, 0.06 minutes
Trial 14 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 15 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 16 of 100
Computing bipartite LCP
99.52 %, 0.05 minutes
Trial 17 of 100
C

In [30]:
# Common neighbors
r = val.cross_val(b_lcp.bipartite_lcp_single, s_nt, links_to_del_s_nt, *('cn',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r, axis=0))
print(np.std(r, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 2 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 3 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 4 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 5 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 6 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 7 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 8 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 9 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 10 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 11 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 12 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 13 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 14 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 15 of 100
Computing bipartite LCP
99.54 %, 0.06 minutes
Trial 16 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 17 of 100
C

In [31]:
# JC
r = val.cross_val(b_lcp.bipartite_lcp_single, s_nt, links_to_del_s_nt, *('jc',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r, axis=0))
print(np.std(r, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 2 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 3 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 4 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 5 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 6 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 7 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 8 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 9 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 10 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 11 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 12 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 13 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 14 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 15 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 16 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 17 of 100
C

In [32]:
# CAR
r = val.cross_val(b_lcp.bipartite_lcp_single, s_nt, links_to_del_s_nt, *('car',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r, axis=0))
print(np.std(r, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 2 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 3 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 4 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 5 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 6 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 7 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 8 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 9 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 10 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 11 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 12 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 13 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 14 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 15 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 16 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 17 of 100
C

In [33]:
# CPA
r = val.cross_val(b_lcp.bipartite_lcp_single, s_nt, links_to_del_s_nt, *('cpa',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r, axis=0))
print(np.std(r, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 2 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 3 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 4 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 5 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 6 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 7 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 8 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 9 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 10 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 11 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 12 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 13 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 14 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 15 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 16 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 17 of 100
C

In [34]:
# RA
r = val.cross_val(b_lcp.bipartite_lcp_single, s_nt, links_to_del_s_nt, *('ra',), 
                   mode='lcp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r, axis=0))
print(np.std(r, axis=0))

Trial 1 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 2 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 3 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 4 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 5 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 6 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 7 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 8 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 9 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 10 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 11 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 12 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 13 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 14 of 100
Computing bipartite LCP
99.54 %, 0.04 minutes
Trial 15 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 16 of 100
Computing bipartite LCP
99.54 %, 0.05 minutes
Trial 17 of 100
C

## Mutual information on bipartite datasets

The mutual information/self information method can also be generalized to the bipartite case. To do so we simply make a slight adjustment to the combinatoric 'a priori' probability, and use the clustering coefficient formulated with triangles instead of quadrangles for the conditional probability. On the previously published datasets, this approach appears to be comparable to LCP, doing slightly better in some cases and slightly worse in others. 

Let us quickly go into more detail on the modified probability definition. Suppose we have two nodes, $a$ and $b$ with degrees $k_a$ and $k_b$. Node $a$ is in class 1 and node $b$ is in class 2. Let class 1 have $N_1$ nodes and class 2 have $N_2$ nodes. Then, using similar logic to the monopartite case, we have that the number of ways for $a$ and $b$ to be connected is 

$$ \binom{N_2-1}{k_a-1}\binom{N_1-1}{k_b-1} $$

The modification comes from the fact that $a$ can only be connected to the $N_2$ nodes in class 2, and vice versa for $b$. We also have that the number of ways for $a$ and $b$ to not be connected is

$$ \binom{N_2-1}{k_a}\binom{N_1-1}{k_b} $$

and the probability follows:

$$ p(L_{ab}|k_a, k_b)  = \frac{\binom{N_2-1}{k_a-1}\binom{N_1-1}{k_b-1}}{\binom{N_2-1}{k_a-1}\binom{N_1-1}{k_b-1}+\binom{N_2-1}{k_a}\binom{N_1-1}{k_b}}$$

As in the monopartite case, there will be an extra counting factor in both terms, but it will cancel when we divide them to compute the probability. 

In [19]:
r7 = val.cross_val(b_si.si_scores, x_e, links_to_del_e, *(probas.proba_comb_links_all, b_mi.mi_lcl_cc), 
                   mode='si_bp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r7, axis=0))
print(np.std(r7, axis=0))

Trial 1 of 100
Computing mutual information
94.99 %, 2.12 minutes
Trial 2 of 100
Computing mutual information
94.99 %, 3.93 minutes
Trial 3 of 100
Computing mutual information
48.99 %, 2.70 minutes
Trial 4 of 100
Computing mutual information
58.99 %, 1.83 minutes
Trial 5 of 100
Computing mutual information
92.99 %, 1.90 minutes
Trial 6 of 100
Computing mutual information
85.99 %, 1.93 minutes
Trial 7 of 100
Computing mutual information
94.99 %, 2.09 minutes
Trial 8 of 100
Computing mutual information
94.99 %, 2.01 minutes
Trial 9 of 100
Computing mutual information
92.99 %, 2.01 minutes
Trial 10 of 100
Computing mutual information
77.99 %, 1.76 minutes
Trial 11 of 100
Computing mutual information
77.99 %, 1.73 minutes
Trial 12 of 100
Computing mutual information
79.99 %, 1.75 minutes
Trial 13 of 100
Computing mutual information
72.99 %, 1.71 minutes
Trial 14 of 100
Computing mutual information
94.99 %, 2.00 minutes
Trial 15 of 100
Computing mutual information
94.99 %, 2.02 minutes
Tria

In [20]:
rr7 = val.cross_val(b_si.si_scores, x_e, links_to_del_e, *(probas.proba_comb_deg_all, b_mi.mi_lcl_cc), 
                   mode='si_bp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(rr7, axis=0))
print(np.std(rr7, axis=0))

Trial 1 of 100
Computing mutual information
94.99 %, 1.96 minutes
Trial 2 of 100
Computing mutual information
94.99 %, 2.11 minutes
Trial 3 of 100
Computing mutual information
48.99 %, 1.18 minutes
Trial 4 of 100
Computing mutual information
58.99 %, 1.39 minutes
Trial 5 of 100
Computing mutual information
92.99 %, 1.90 minutes
Trial 6 of 100
Computing mutual information
85.99 %, 1.91 minutes
Trial 7 of 100
Computing mutual information
94.99 %, 2.04 minutes
Trial 8 of 100
Computing mutual information
94.99 %, 1.98 minutes
Trial 9 of 100
Computing mutual information
92.99 %, 2.00 minutes
Trial 10 of 100
Computing mutual information
77.99 %, 1.84 minutes
Trial 11 of 100
Computing mutual information
77.99 %, 1.79 minutes
Trial 12 of 100
Computing mutual information
79.99 %, 1.77 minutes
Trial 13 of 100
Computing mutual information
72.99 %, 1.73 minutes
Trial 14 of 100
Computing mutual information
94.99 %, 2.02 minutes
Trial 15 of 100
Computing mutual information
94.99 %, 2.06 minutes
Tria

In [21]:
r8 = val.cross_val(b_si.si_scores, x_g, links_to_del_g, *(probas.proba_comb_links_all, b_mi.mi_lcl_cc), 
                   mode='si_bp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r8, axis=0))
print(np.std(r8, axis=0))

Trial 1 of 100
Computing mutual information
59.96 %, 0.20 minutes
Trial 2 of 100
Computing mutual information
75.95 %, 0.22 minutes
Trial 3 of 100
Computing mutual information
58.96 %, 0.17 minutes
Trial 4 of 100
Computing mutual information
75.95 %, 0.21 minutes
Trial 5 of 100
Computing mutual information
93.94 %, 0.22 minutes
Trial 6 of 100
Computing mutual information
93.94 %, 0.21 minutes
Trial 7 of 100
Computing mutual information
84.95 %, 0.20 minutes
Trial 8 of 100
Computing mutual information
75.95 %, 0.20 minutes
Trial 9 of 100
Computing mutual information
93.94 %, 0.20 minutes
Trial 10 of 100
Computing mutual information
58.96 %, 0.18 minutes
Trial 11 of 100
Computing mutual information
84.95 %, 0.20 minutes
Trial 12 of 100
Computing mutual information
84.95 %, 0.20 minutes
Trial 13 of 100
Computing mutual information
84.95 %, 0.21 minutes
Trial 14 of 100
Computing mutual information
58.96 %, 0.18 minutes
Trial 15 of 100
Computing mutual information
84.95 %, 0.20 minutes
Tria

In [22]:
rr8 = val.cross_val(b_si.si_scores, x_g, links_to_del_g, *(probas.proba_comb_deg_all, b_mi.mi_lcl_cc), 
                   mode='si_bp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(rr8, axis=0))
print(np.std(rr8, axis=0))

Trial 1 of 100
Computing mutual information
59.96 %, 0.19 minutes
Trial 2 of 100
Computing mutual information
75.95 %, 0.20 minutes
Trial 3 of 100
Computing mutual information
58.96 %, 0.17 minutes
Trial 4 of 100
Computing mutual information
75.95 %, 0.21 minutes
Trial 5 of 100
Computing mutual information
93.94 %, 0.20 minutes
Trial 6 of 100
Computing mutual information
93.94 %, 0.20 minutes
Trial 7 of 100
Computing mutual information
84.95 %, 0.20 minutes
Trial 8 of 100
Computing mutual information
75.95 %, 0.20 minutes
Trial 9 of 100
Computing mutual information
93.94 %, 0.20 minutes
Trial 10 of 100
Computing mutual information
58.96 %, 0.17 minutes
Trial 11 of 100
Computing mutual information
84.95 %, 0.19 minutes
Trial 12 of 100
Computing mutual information
84.95 %, 0.20 minutes
Trial 13 of 100
Computing mutual information
84.95 %, 0.23 minutes
Trial 14 of 100
Computing mutual information
58.96 %, 0.20 minutes
Trial 15 of 100
Computing mutual information
84.95 %, 0.26 minutes
Tria

In [23]:
r9 = val.cross_val(b_si.si_scores, x_ic, links_to_del_ic, *(probas.proba_comb_links_all, b_mi.mi_lcl_cc), 
                   mode='si_bp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r9, axis=0))
print(np.std(r9, axis=0))

Trial 1 of 100
Computing mutual information
98.97 %, 1.05 minutes
Trial 2 of 100
Computing mutual information
98.97 %, 1.02 minutes
Trial 3 of 100
Computing mutual information
96.97 %, 1.02 minutes
Trial 4 of 100
Computing mutual information
97.97 %, 1.02 minutes
Trial 5 of 100
Computing mutual information
98.97 %, 1.00 minutes
Trial 6 of 100
Computing mutual information
96.97 %, 0.96 minutes
Trial 7 of 100
Computing mutual information
98.97 %, 1.02 minutes
Trial 8 of 100
Computing mutual information
97.97 %, 1.03 minutes
Trial 9 of 100
Computing mutual information
97.97 %, 1.03 minutes
Trial 10 of 100
Computing mutual information
97.97 %, 1.01 minutes
Trial 11 of 100
Computing mutual information
97.97 %, 1.02 minutes
Trial 12 of 100
Computing mutual information
98.97 %, 1.07 minutes
Trial 13 of 100
Computing mutual information
98.97 %, 1.07 minutes
Trial 14 of 100
Computing mutual information
98.97 %, 1.04 minutes
Trial 15 of 100
Computing mutual information
95.97 %, 0.93 minutes
Tria

In [24]:
rr9 = val.cross_val(b_si.si_scores, x_ic, links_to_del_ic, *(probas.proba_comb_deg_all, b_mi.mi_lcl_cc), 
                   mode='si_bp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(rr9, axis=0))
print(np.std(rr9, axis=0))

Trial 1 of 100
Computing mutual information
98.97 %, 1.04 minutes
Trial 2 of 100
Computing mutual information
98.97 %, 1.02 minutes
Trial 3 of 100
Computing mutual information
96.97 %, 1.02 minutes
Trial 4 of 100
Computing mutual information
97.97 %, 1.02 minutes
Trial 5 of 100
Computing mutual information
98.97 %, 1.01 minutes
Trial 6 of 100
Computing mutual information
96.97 %, 0.97 minutes
Trial 7 of 100
Computing mutual information
98.97 %, 1.09 minutes
Trial 8 of 100
Computing mutual information
97.97 %, 1.04 minutes
Trial 9 of 100
Computing mutual information
97.97 %, 1.03 minutes
Trial 10 of 100
Computing mutual information
97.97 %, 1.01 minutes
Trial 11 of 100
Computing mutual information
97.97 %, 1.02 minutes
Trial 12 of 100
Computing mutual information
98.97 %, 1.07 minutes
Trial 13 of 100
Computing mutual information
98.97 %, 1.07 minutes
Trial 14 of 100
Computing mutual information
98.97 %, 1.04 minutes
Trial 15 of 100
Computing mutual information
95.97 %, 0.94 minutes
Tria

## Mutual information on bipartite Elegans dataset

When we move back to the Elegans dataset we see a significant improvement in the predictive power of mutual information over LCP. The reason for this is unclear. 

In [25]:
r10 = val.cross_val(b_si.si_scores, s_nt, links_to_del_s_nt, *(probas.proba_comb_links_all, b_mi.mi_lcl_cc), 
                   mode='si_bp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r10, axis=0))
print(np.std(r10, axis=0))

Trial 1 of 100
Computing mutual information
99.54 %, 0.28 minutes
Trial 2 of 100
Computing mutual information
99.54 %, 0.28 minutes
Trial 3 of 100
Computing mutual information
99.54 %, 0.29 minutes
Trial 4 of 100
Computing mutual information
99.54 %, 0.27 minutes
Trial 5 of 100
Computing mutual information
99.54 %, 0.28 minutes
Trial 6 of 100
Computing mutual information
98.55 %, 0.28 minutes
Trial 7 of 100
Computing mutual information
99.54 %, 0.30 minutes
Trial 8 of 100
Computing mutual information
99.54 %, 0.29 minutes
Trial 9 of 100
Computing mutual information
99.54 %, 0.38 minutes
Trial 10 of 100
Computing mutual information
95.56 %, 0.28 minutes
Trial 11 of 100
Computing mutual information
98.55 %, 0.28 minutes
Trial 12 of 100
Computing mutual information
98.55 %, 0.28 minutes
Trial 13 of 100
Computing mutual information
96.56 %, 0.28 minutes
Trial 14 of 100
Computing mutual information
98.55 %, 0.28 minutes
Trial 15 of 100
Computing mutual information
96.56 %, 0.28 minutes
Tria

In [26]:
r11 = val.cross_val(b_si.si_scores, s_nt, links_to_del_s_nt, *(probas.proba_comb_deg_all, b_mi.mi_lcl_cc), 
                   mode='si_bp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r11, axis=0))
print(np.std(r11, axis=0))

Trial 1 of 100
Computing mutual information
99.54 %, 0.28 minutes
Trial 2 of 100
Computing mutual information
99.54 %, 0.27 minutes
Trial 3 of 100
Computing mutual information
99.54 %, 0.26 minutes
Trial 4 of 100
Computing mutual information
99.54 %, 0.26 minutes
Trial 5 of 100
Computing mutual information
99.54 %, 0.27 minutes
Trial 6 of 100
Computing mutual information
98.55 %, 0.27 minutes
Trial 7 of 100
Computing mutual information
99.54 %, 0.28 minutes
Trial 8 of 100
Computing mutual information
99.54 %, 0.28 minutes
Trial 9 of 100
Computing mutual information
99.54 %, 0.27 minutes
Trial 10 of 100
Computing mutual information
95.56 %, 0.26 minutes
Trial 11 of 100
Computing mutual information
98.55 %, 0.27 minutes
Trial 12 of 100
Computing mutual information
98.55 %, 0.26 minutes
Trial 13 of 100
Computing mutual information
96.56 %, 0.26 minutes
Trial 14 of 100
Computing mutual information
98.55 %, 0.27 minutes
Trial 15 of 100
Computing mutual information
96.56 %, 0.27 minutes
Tria

## Mutual information on tripartite Elegans dataset

Having adapted the mutual information approach for bipartite networks, it turns out to be trivial to predict on tripartite networks, since an undirected bipartite network can actually be thought of as a tripartite network where the first layer maps class 'a' nodes to class 'b' nodes, and the second does the opposite. The nontrivial case is simply when the two layers contain different nodes and edges, but the approach is exactly the same. Unfortunately it appears that there is less useful information in layers that are further removed from the links we are trying to predict, since there is a significant drop in predictive power when applying mutual information to the tripartite Elegans network of source-nt-nr.

In [27]:
r12 = val.cross_val(b_si.si_scores, s_nt, links_to_del_s_nt, *(probas.proba_comb_links_all, b_mi.mi_lcl_cc), 
                   y=nt_nr, mode='si_tp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r12, axis=0))
print(np.std(r12, axis=0))

Trial 1 of 100
Computing mutual information
82.62 %, 0.01 minutes
Trial 2 of 100
Computing mutual information
72.67 %, 0.01 minutes
Trial 3 of 100
Computing mutual information
72.67 %, 0.01 minutes
Trial 4 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 5 of 100
Computing mutual information
72.67 %, 0.01 minutes
Trial 6 of 100
Computing mutual information
82.62 %, 0.01 minutes
Trial 7 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 8 of 100
Computing mutual information
65.70 %, 0.01 minutes
Trial 9 of 100
Computing mutual information
72.67 %, 0.01 minutes
Trial 10 of 100
Computing mutual information
81.62 %, 0.01 minutes
Trial 11 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 12 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 13 of 100
Computing mutual information
64.70 %, 0.01 minutes
Trial 14 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 15 of 100
Computing mutual information
82.62 %, 0.01 minutes
Tria

In [28]:
rr12 = val.cross_val(b_si.si_scores, s_nt, links_to_del_s_nt, *(probas.proba_comb_deg_all, b_mi.mi_lcl_cc), 
                   y=nt_nr, mode='si_tp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(rr12, axis=0))
print(np.std(rr12, axis=0))

Trial 1 of 100
Computing mutual information
82.62 %, 0.01 minutes
Trial 2 of 100
Computing mutual information
72.67 %, 0.01 minutes
Trial 3 of 100
Computing mutual information
72.67 %, 0.01 minutes
Trial 4 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 5 of 100
Computing mutual information
72.67 %, 0.01 minutes
Trial 6 of 100
Computing mutual information
82.62 %, 0.01 minutes
Trial 7 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 8 of 100
Computing mutual information
65.70 %, 0.01 minutes
Trial 9 of 100
Computing mutual information
72.67 %, 0.01 minutes
Trial 10 of 100
Computing mutual information
81.62 %, 0.01 minutes
Trial 11 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 12 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 13 of 100
Computing mutual information
64.70 %, 0.01 minutes
Trial 14 of 100
Computing mutual information
98.55 %, 0.01 minutes
Trial 15 of 100
Computing mutual information
82.62 %, 0.01 minutes
Tria

## LCLs on Tripartite Elegans Data

We can also confirm that the LCP is not as effective when used on tripartite networks, and also that it is still less effective than mutual information for the Elegans dataset. 

In [29]:
r13 = val.cross_val(b_lcp.tripartite_lcls, s_nt, links_to_del_s_nt, (True,),
                   y=nt_nr, mode='lcp_tp', loops=loops, verbose=verbose, plot=plot)
print(np.mean(r13, axis=0))
print(np.std(r13, axis=0))

Trial 1 of 100
Trial 2 of 100
Trial 3 of 100
Trial 4 of 100
Trial 5 of 100
Trial 6 of 100
Trial 7 of 100
Trial 8 of 100
Trial 9 of 100
Trial 10 of 100
Trial 11 of 100
Trial 12 of 100
Trial 13 of 100
Trial 14 of 100
Trial 15 of 100
Trial 16 of 100
Trial 17 of 100
Trial 18 of 100
Trial 19 of 100
Trial 20 of 100
Trial 21 of 100
Trial 22 of 100
Trial 23 of 100
Trial 24 of 100
Trial 25 of 100
Trial 26 of 100
Trial 27 of 100
Trial 28 of 100
Trial 29 of 100
Trial 30 of 100
Trial 31 of 100
Trial 32 of 100
Trial 33 of 100
Trial 34 of 100
Trial 35 of 100
Trial 36 of 100
Trial 37 of 100
Trial 38 of 100
Trial 39 of 100
Trial 40 of 100
Trial 41 of 100
Trial 42 of 100
Trial 43 of 100
Trial 44 of 100
Trial 45 of 100
Trial 46 of 100
Trial 47 of 100
Trial 48 of 100
Trial 49 of 100
Trial 50 of 100
Trial 51 of 100
Trial 52 of 100
Trial 53 of 100
Trial 54 of 100
Trial 55 of 100
Trial 56 of 100
Trial 57 of 100
Trial 58 of 100
Trial 59 of 100
Trial 60 of 100
Trial 61 of 100
Trial 62 of 100
Trial 63 of 100
T